In [ ]:
#so given a specific uhhhh uhhh pixel we wanna randomize values of 4 seperate pixels that average to that pixel? we can probably brute force this
#for now if we just wanna do the string of encrypted pixels from the previous program but will need to be reworked to have a smarter
#algorithm if we wanna scale up > will do if we have time

#imports
import random
import PIL
from PIL import Image 
from pathlib import Path
desnoise = [128,128,128,255] #des rgb val for noise
desnoise[0] = int(input("Desired noise red val?"))
desnoise[1] = int(input("Desired noise green val?"))
desnoise[2] = int(input("Desired noise blue val?"))
desbuffer = int(input("Desired noise buffer? (Rec: 5)"))
dessignal = [160,128,128,255] #des rgb val for signal
dessignal[0] = int(input("Desired signal red val?"))
dessignal[1] = int(input("Desired signal green val?"))
dessignal[2] = int(input("Desired signal blue val?"))
def expand_pixel(des = [128,128,128,255], noise_buffer = 5): #should be passed an array with length 4 desired containing r,g,b,a values, default white because its easier to average to 255 and you cant really avg to black
    #note to matias itf we might have to use a secondary color as the desired bg, not black.
    #noise buffer is how exact we want the generated pixel to be compared to the ideal, ie what gets let through
    #higher number = less exact vals, but faster gen times, and more noisey noise
    des_red = des[0]
    des_green = des[1]
    des_blue = des[2]
    des_alpha = des[3]
    #print("Expanding pixel with RGBA values: " + str(des_red) + " " + str(des_blue) + " " + str(des_green) + " " + str(des_alpha))
    
    #so we have our desired r value (255)???? so then we do a super inefficient while loop because im evil
    r1 = 0
    r2 = 0
    r3 = 0
    r4 = 0
    while ((r1+r2+r3+r4)/4) < (des_red - noise_buffer) or ((r1+r2+r3+r4)/4) > (des_red + noise_buffer):
        r1 = random.randint(0,255)
        r2 = random.randint(0,255)
        r3 = random.randint(0,255)
        r4 = random.randint(0,255)  
    #print("Values chosen for red. Values chosen: " + str(r1) + " "+ str(r2) + " "+ str(r3) + " "+ str(r4) + ", averaging to " + str((r1+r2+r3+r4)/4) )
    
    #disgusting evil expanded code
    g1 = 0
    g2 = 0
    g3 = 0
    g4 = 0
    while ((g1+g2+g3+g4)/4) < (des_green - noise_buffer) or ((g1+g2+g3+g4)/4) > (des_green + noise_buffer):
        g1 = random.randint(0,255)
        g2 = random.randint(0,255)
        g3 = random.randint(0,255)
        g4 = random.randint(0,255)  
    #print("Values chosen for green. Values chosen: " + str(g1) + " "+ str(g2) + " "+ str(g3) + " "+ str(g4) + ", averaging to " + str((g1+g2+g3+g4)/4) )
    b1 = 0
    b2 = 0
    b3 = 0
    b4 = 0
    while ((b1+b2+b3+b4)/4) < (des_blue - noise_buffer) or ((b1+b2+b3+b4)/4) > (des_blue + noise_buffer):
        b1 = random.randint(0,255)
        b2 = random.randint(0,255)
        b3 = random.randint(0,255)
        b4 = random.randint(0,255)  
    #print("Values chosen for blue. Values chosen: " + str(b1) + " "+ str(b2) + " "+ str(b3) + " "+ str(b4) + ", averaging to " + str((b1+b2+b3+b4)/4) )

    return [[r1, r2, r3, r4],[g1,g2,g3,g4],[b1,b2,b3,b4]]



# #image stuffs, create our noisy/uncompressed background based on given ideal RGBA values and tolerance
original_size = 16#size of source image to be expanded
expansion_scale_factor = 2 #decompression factor, width of expanded pixel
des_size = original_size * expansion_scale_factor
input_image = Image.new(mode="RGBA", size=(des_size, des_size),
                        color="black")
pixel_map = input_image.load() #loads the image into a pixel map
width, height = input_image.size #gets the width and height of the image
for i in range(original_size):
   for j in range(original_size): 
        [[r1,r2,r3,r4],[g1,g2,g3,g4],[b1,b2,b3,b4]] = expand_pixel(desnoise, desbuffer) #change values here until i fix it and make it swag
        #working with a singular pixel0
        #top left
        pixel_map[2*i, 2*j] = (r1,g1,b1,255)
        #top right
        pixel_map[2*i+1, 2*j] = (r2,g2,b2,255)
        #bottom left
        pixel_map[2*i, 2*j+1] = (r3,g3,b3,255)
        #bottom right
        pixel_map[2*i+1, 2*j+1] = (r4,g4,b4,255)




#so now we need to generate similar noise but slightly offset based on an input image, it might be easier to expand pixel noise based on like 8x8 input image in b/w
input_filename = Path(input("Enter stencil filepath (DO NOT INCLUDE QUOTES): ")) #filepath for our "stencil" image
stencil = PIL.Image.open(input_filename) #opens our image for reading in code
st_width, st_height = stencil.size #gets the width and height of the image
black_pixels=[] #2d array for storing our black pixel positions (positions to use in pixel expansion)
pixels = list(stencil.getdata()) #loads our pixel data
#now we need to process the data in a 2d array?? or get xy posiitons so i guess we split into arrays of width
held = []
for i in range(st_height):
    held = [] 
    for j in range(st_width):
        held.append(pixels[i*st_width + j]) #this might not work but im evil evil
    black_pixels.append(held)
#print(black_pixels)

#now we go through and run expand pixel on our noisy image based on the values holding black pixels but for now we can try just appending onto noisy image to get it working
coords = []
activecoord = []
for i in range(len(black_pixels)):
    for j in range(len(black_pixels[i])):
        activecoord=[j,i]
        if black_pixels[i][j] == 1:
            coords.append(activecoord)
print(coords)


noisey = Path(input("Enter filepath of generated noise file (NO QUOTES): "))
noiseymap = PIL.Image.open(noisey)
noiseymap = noiseymap.load()
for i in range(len(coords)):
    rgbs = expanded_vals = expand_pixel(dessignal, 5) #generates a set of 4 rgb vals as per expand pixel func
    
    #probably a smarter way to do this but i dont really care
    pixel_map[2*coords[i][0], 2*coords[i][1]] = (rgbs[0][0], rgbs[1][0], rgbs[2][0], 255)
    pixel_map[2*coords[i][0]+1, 2*coords[i][1]] = (rgbs[0][1], rgbs[1][1], rgbs[2][1], 255)
    pixel_map[2*coords[i][0], 2*coords[i][1]+1] = (rgbs[0][2], rgbs[1][2], rgbs[2][2], 255)
    pixel_map[2*coords[i][0]+1, 2*coords[i][1]+1] = (rgbs[0][3], rgbs[1][3], rgbs[2][3], 255)


input_image.save("output.png", format="png") 
input_image.show()

[[6, 1], [7, 1], [8, 1], [5, 2], [9, 2], [4, 3], [10, 3], [12, 3], [3, 4], [4, 4], [5, 4], [10, 4], [11, 4], [13, 4], [2, 5], [6, 5], [10, 5], [13, 5], [3, 6], [4, 6], [5, 6], [10, 6], [13, 6], [4, 7], [10, 7], [13, 7], [4, 8], [10, 8], [13, 8], [4, 9], [10, 9], [11, 9], [13, 9], [4, 10], [10, 10], [12, 10], [4, 11], [7, 11], [10, 11], [4, 12], [7, 12], [10, 12], [3, 13], [7, 13], [11, 13], [4, 14], [5, 14], [6, 14], [7, 14], [8, 14], [9, 14], [10, 14]]
